### 생성 데이터 컬럼
> 위치 및 충전기 정보
- 충전소위치  충전기 <br/>	
> 충전기 정보 상세
- 충전기타입  충전기상태  충전기용량  방전지원여부  충전량  요청충전량  방전량  요청방전량<br>  
> 시간 정보
- 연결시작시간  연결종료시간  충전시작시간  충전종료시간  출방예상시간<br/>
> 차량 Soc 정보
- 베터리용량  시작값  종료값  연비

In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta,timezone
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import gaussian_kde
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [4]:
np.random.seed(42)
random.seed(42)

evse_dict = defaultdict(list)
#충전소위치
st_number = 50  # 충전소 개수
locations = [f'st-{str(n).zfill(2)}' for n in range(st_number)]
#충전기 Electric Vehicle Supply Equipment
evse_info = {}
evse_types = ['FC', 'SC']
fc_evse_volume_list = [50, 100, 200]
sc_evse_volume_list = [10, 30, 50]
evse_names = []

for loc in locations:
    evse_n = random.randint(2,6)
    for i in range(evse_n):
        evse_name = f'{loc}_evse-{str(i).zfill(2)}'
        evse_type = random.choices(['FC', 'SC'], weights=[0.3, 0.7])[0]
        if evse_type =='FC':
            volume = random.choice(fc_evse_volume_list)
        else:
            volume = random.choice(sc_evse_volume_list)
            
        evse_names.append(evse_name)
        evse_dict[loc].append(evse_name)
        evse_info[evse_name] = {
            'type': evse_type,
            'volume': volume
        }
evse_dict = defaultdict(list)

for evse in evse_names:
    loc = evse.split('_')[0]
    evse_dict[loc].append(evse)

evse_status_list = ['Available']

yes_no = ['y', 'n']
battery_capacities = [50, 60, 75, 90, 100]
TIME_LIMIT = datetime(2024, 12, 31, 23, 59, tzinfo=timezone.utc)

#Soc상태 생성
def generate_battery_percent(start=True, delivered_energy=None, capacity=100):
    if start:
        return round(np.clip(np.random.beta(2, 5) * 100, 5, 60), 1)
    else:
        if delivered_energy is None:
            return round(np.clip(np.random.beta(5, 2) * 100, 30, 100), 1)
        # 종료 배터리 퍼센트 = 시작 + 충전량 / 용량 * 100 (대략)
        increase = min((delivered_energy / capacity) * 100 + np.random.normal(0, 3), 100)
        return round(np.clip(increase, 5, 100), 1)

#충전량 요청
def generate_requested_energy():
    if np.random.rand() < 0.008:  # 예:0.5% 확률로 0 생성
        return 0.0
    val = np.random.lognormal(mean=2.98, sigma=0.4)  # 중앙값 약 10~30 부근
    return round(val, 2)

#충전량
def generate_energy_usage(evse_volume,request_energy):
    if request_energy == 0: #충전요청이 없을 경우
        if np.random.rand() < 0.2: # 2퍼 확률로 충전전류 생성
            return round(random.uniform(0,10), 2) # 충전 요청 없었지만 충전된 경우
        return  0
    offset = request_energy*0.037 #0.37프로 범위내로 +- 존재하게 만듬
    return round(random.uniform(request_energy-offset,request_energy + offset),2)

#방전량
def generate_discharge(supports_discharge):
    if supports_discharge == 'n':
        return 0, 0
    discharged = int(np.random.gamma(2, 5))
    requested = discharged + random.randint(-3, 5)
    return max(discharged, 0), max(requested, 0)

#충전타입에 따른 충전시간 생성
def generate_charge_duration(evse_type, delivered_energy):
    """
    충전량(kWh)과 충전기 속도에 기반해 충전 시간(timedelta) 계산
    - 충전기 타입에 따라 충전 속도 (kW) 설정
    - 최소 시간은 soft limit으로 적용
    """
    if delivered_energy == 0:
        return timedelta(minutes=0)

    # 평균 속도
    if evse_type == 'FC':
        charge_speed = random.uniform(50, 80)
        soft_min_minutes = 5
    else:
        charge_speed = random.uniform(3.3, 7.2)
        soft_min_minutes = 10

    # 이상적인 충전 시간
    charge_time_minutes = (delivered_energy / charge_speed) * 60

    # ✅ 1. 효율성 저하 반영 (예: 충전속도 저하 등)
    inefficiency_factor = np.random.normal(1.05, 0.05)  # 5~10% 더 걸리도록
    charge_time_minutes *= inefficiency_factor

    # ✅ 2. 부드러운 최소 시간 보장 (hard max 대신 soft push)
    # charge_time_minutes가 너무 작으면 soft_min_minutes와 중간값으로 섞음
    alpha = np.clip((charge_time_minutes - soft_min_minutes) / soft_min_minutes, 0, 1)
    adjusted_minutes = (1 - alpha) * soft_min_minutes + alpha * charge_time_minutes

    # ✅ 3. jitter (±2분)
    jitter = np.random.normal(0, 2)
    final_minutes = round(np.clip(adjusted_minutes + jitter, soft_min_minutes, 180))

    return timedelta(minutes=final_minutes)

#충전기별 빈도 가중치 생성
evse_popularity_score = {}
def get_evse_popularity(evse_name):
    if evse_name not in evse_popularity_score:
        # 평균 0.4~0.6 중심, 일부는 높고 일부는 낮게
        evse_popularity_score[evse_name] = np.clip(np.random.beta(2, 3), 0.05, 0.95) # 베타 분포
    return evse_popularity_score[evse_name]

#빈도 가중치별 간격 생성
def generate_idle_gap_with_demand(evse_name, current_time):
    popularity = get_evse_popularity(evse_name)
    base_gap = 600 - int(popularity * 550)  # 기본 gap

    # 시간대별 방문 가능성
    hour = current_time.hour
    if 7 <= hour <= 9 or 17 <= hour <= 19:
        demand_factor = 0.8  # 출퇴근 시간엔 방문 많음
    elif 1 <= hour <= 5:
        demand_factor = 0.2  # 심야엔 방문 적음
    else:
        demand_factor = 0.5  # 평상시

    # 방문 안하는 확률 (ex: 인기 낮으면 방문 안할 확률 증가)
    no_visit_prob = max(0, 1 - popularity * demand_factor)
    if random.random() < no_visit_prob:
        # 긴 텀 생성 (1~3일)
        return base_gap + random.randint(6*60, 3*24*60) #6시간에서 3일 정도 방문 안함
    else:
        # 짧은 텀 + 약간 노이즈
        noise = random.randint(-20, 20)
        return max(base_gap + noise, 10)

#이전 충전기록 저장
evse_last_end_time = {}
#시간관련 정보생성
def generate_times(evse_type, evse_name, requested_energy, delivered_energy):
    # 마지막 종료시간 이후 시작
    last_end = evse_last_end_time.get(evse_name, datetime(2019, 11, 1, tzinfo=timezone.utc))
    # 빈도에 따른 간격 생성
    idle_gap_minutes = generate_idle_gap_with_demand(evse_name, last_end)
    # 간격 반영한 새로운 충전 시간 생성
    start = last_end + timedelta(minutes=idle_gap_minutes)
    
    if start>TIME_LIMIT :
        raise StopIteration
    
    charge_delay = timedelta(minutes=random.randint(1,30))
    charge_duration = generate_charge_duration(evse_type, delivered_energy)
    
    # 연결 유지 시간 (충전 종료 후 출발까지 남아 있는 시간 등 포함)
    extra_seconds = np.clip(np.random.lognormal(mean=7.5, sigma=0.7), 300, 36000)
    conn_duration = charge_delay + charge_duration + timedelta(seconds=round(extra_seconds))

    if requested_energy == 0:
        chg_start, chg_end = None, None
    else:
        chg_start = start + charge_delay
        chg_end = chg_start + charge_duration

    conn_end = start + conn_duration
    evse_last_end_time[evse_name] = conn_end

    return start, conn_end, chg_start, chg_end, last_end

#출발예상시간 생성
def generate_departure(chg_start,chg_end):
    if chg_end is None:
        return None  # 충전 종료시간이 없으면 출발예상시간도 없음
    if random.random() < 0.15:
        return None  # 출발예상시간 미입력 케이스

    if random.random() < 0.8:
        # ✅ 80%: 충전 종료 이후 출발 (자연스러운 분포)
        choice = random.random()
        if choice < 0.6:
            delay = np.random.normal(20, 10)  # 일반적인 경우
        elif choice < 0.85:
            delay = np.random.exponential(30)  # 30분 지수분포
        else:
            delay = np.random.normal(45, 15)  # 길게 대기
        delay = round(np.clip(delay, 1, 120))  # 1~120분 제한
        return chg_end + timedelta(minutes=delay)
    else:
        # ✅ 20%: 충전 종료 전 출발 예정 (단, 충전 시작보다 이전이면 안됨)
        for _ in range(10):  # 최대 10번 반복
            choice = random.random()
            if choice < 0.7:
                advance = np.random.normal(10, 5)
            else:
                advance = np.random.exponential(15)
            advance = round(np.clip(advance, 1, 60))
            proposed_time = chg_end - timedelta(minutes=advance)
            if proposed_time >= chg_start:
                return proposed_time
        
        # fallback: 실패하면 충전 종료 직전으로 설정
        return chg_start + timedelta(minutes=1)

#데이터 생성
def generate_dataframe(evse_dict, n=200):
    data = []
    for location, evse_list in evse_dict.items():
        for evse_name in evse_list:
            
            for _ in range(n):
                # evse_name = random.choice(evse_names) 
                location = evse_name.split('_')[0]  
                evse_type = evse_info[evse_name]['type']
                evse_status = 'Available'
                evse_volume = evse_info[evse_name]['volume']
                discharge_supported = 'y' if evse_type == 'SC' and random.random() < 0.5 else 'n'
                scheduled_charge = random.choices(yes_no, weights=[0.3, 0.7])[0]

                
                requested_energy = generate_requested_energy()
                delivered_energy = generate_energy_usage(evse_volume,requested_energy)
                battery_capacity = random.choice(battery_capacities)
                battery_start = generate_battery_percent(start=True, capacity=battery_capacity)
                battery_end = generate_battery_percent(start=False, delivered_energy=delivered_energy, capacity=battery_capacity)

                discharge, requested_discharge = generate_discharge(discharge_supported)
                efficiency = round(np.clip(np.random.normal(5.5, 0.8), 3.5, 7.5), 2)
                try:
                    conn_start, conn_end, chg_start, chg_end, last_end = generate_times(evse_type,evse_name,requested_energy,delivered_energy)
                except StopIteration:
                    break
                departure_time = generate_departure(chg_start,chg_end)
                
                
                data.append({
                    '충전소위치': location,
                    '충전기이름': evse_name,
                    '충전기타입': evse_type,
                    '충전기상태': evse_status,
                    '충전기용량': evse_volume,
                    '방전지원여부': discharge_supported,
                    '예약충전': scheduled_charge,
                    '충전량(kWh)': delivered_energy,
                    '요청충전량(kWh)': requested_energy,
                    '방전량(kWh)': discharge,
                    '요청방전량(kWh)': requested_discharge,
                    '마지막충전종료시간': last_end.isoformat(),
                    '연결시작시간': conn_start.isoformat(),
                    '충전시작시간': chg_start.isoformat() if chg_start else None,
                    '충전종료시간': chg_end.isoformat() if chg_end else None,
                    '연결종료시간': conn_end.isoformat(),
                    '출발예상시간': departure_time.isoformat() if departure_time else None,
                    '베터리용량(kWh)': battery_capacity,
                    '시작베터리%': battery_start,
                    '종료베터리%': battery_end,
                    '연비(Wh/km)': efficiency
                })
    return pd.DataFrame(data)

df_clean = generate_dataframe(evse_dict= evse_dict, n=2000)
df_clean

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
0,st-00,st-00_evse-00,FC,Available,100,n,n,12.83,12.62,0,...,2019-11-01T00:00:00+00:00,2019-11-03T00:32:00+00:00,2019-11-03T00:36:00+00:00,2019-11-03T00:50:00+00:00,2019-11-03T01:00:22+00:00,2019-11-03T00:55:00+00:00,75,19.1,19.4,5.04
1,st-00,st-00_evse-00,FC,Available,100,n,y,15.33,14.86,0,...,2019-11-03T01:00:22+00:00,2019-11-06T06:49:22+00:00,2019-11-06T07:13:22+00:00,2019-11-06T07:25:22+00:00,2019-11-06T08:23:51+00:00,2019-11-06T08:14:22+00:00,100,5.0,11.1,5.60
2,st-00,st-00_evse-00,FC,Available,100,n,n,15.41,15.27,0,...,2019-11-06T08:23:51+00:00,2019-11-07T18:31:51+00:00,2019-11-07T18:48:51+00:00,2019-11-07T19:01:51+00:00,2019-11-07T19:13:45+00:00,2019-11-07T19:23:51+00:00,50,37.6,33.3,4.52
3,st-00,st-00_evse-00,FC,Available,100,n,y,21.36,21.49,0,...,2019-11-07T19:13:45+00:00,2019-11-10T22:28:45+00:00,2019-11-10T22:34:45+00:00,2019-11-10T22:50:45+00:00,2019-11-10T23:07:01+00:00,2019-11-10T22:59:45+00:00,90,41.1,22.8,6.12
4,st-00,st-00_evse-00,FC,Available,100,n,y,20.67,20.78,0,...,2019-11-10T23:07:01+00:00,2019-11-12T10:44:01+00:00,2019-11-12T10:59:01+00:00,2019-11-12T11:18:01+00:00,2019-11-12T11:56:01+00:00,2019-11-12T11:30:01+00:00,100,13.3,23.8,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212521,st-49,st-49_evse-04,FC,Available,200,n,y,21.35,21.73,0,...,2024-12-18T17:02:01+00:00,2024-12-21T02:48:01+00:00,2024-12-21T03:01:01+00:00,2024-12-21T03:21:01+00:00,2024-12-21T03:59:09+00:00,2024-12-21T03:10:01+00:00,90,60.0,26.9,6.20
212522,st-49,st-49_evse-04,FC,Available,200,n,y,11.33,11.22,0,...,2024-12-21T03:59:09+00:00,2024-12-24T09:09:09+00:00,2024-12-24T09:10:09+00:00,2024-12-24T09:22:09+00:00,2024-12-24T10:46:57+00:00,2024-12-24T09:13:09+00:00,100,51.6,17.9,6.57
212523,st-49,st-49_evse-04,FC,Available,200,n,n,18.77,18.48,0,...,2024-12-24T10:46:57+00:00,2024-12-27T13:12:57+00:00,2024-12-27T13:24:57+00:00,2024-12-27T13:37:57+00:00,2024-12-27T13:46:56+00:00,2024-12-27T14:06:57+00:00,75,25.9,28.5,6.44
212524,st-49,st-49_evse-04,FC,Available,200,n,n,16.83,16.32,0,...,2024-12-27T13:46:56+00:00,2024-12-27T21:31:56+00:00,2024-12-27T21:39:56+00:00,2024-12-27T22:02:56+00:00,2024-12-27T22:53:14+00:00,2024-12-27T21:54:56+00:00,75,52.1,19.9,4.51


In [5]:
df_clean.to_csv('../csv/50area_dummy1_232,000.csv',index=False)

In [185]:
df_clean.duplicated(keep=False).sum()

0

In [168]:
start = pd.to_datetime(df_clean['연결시작시간'],format='ISO8601')
last_end = pd.to_datetime(df_clean['마지막충전종료시간'],format='ISO8601')

delta = start-last_end
mask = delta > pd.Timedelta(days=0)
delta

0       2 days 02:38:00
1       3 days 07:13:00
2       3 days 02:24:00
3       3 days 05:26:00
4       2 days 08:02:00
              ...      
23195   2 days 19:05:00
23196   1 days 10:44:00
23197   1 days 03:11:00
23198   0 days 23:21:00
23199   1 days 04:47:00
Length: 23200, dtype: timedelta64[ns]

In [69]:
df = df_clean[['충전기타입','요청충전량(kWh)','충전시작시간','충전종료시간']]
df[df['요청충전량(kWh)'].between(10, 15)]

,충전기타입,요청충전량(kWh),충전시작시간,충전종료시간
0,SC,10.54,2019-11-01T07:50:00+00:00,2019-11-01T09:17:00+00:00
1,SC,11.44,2019-11-01T08:04:00+00:00,2019-11-01T09:58:00+00:00
2,FC,15.00,2019-11-01T05:07:00+00:00,2019-11-01T05:26:00+00:00
8,SC,13.89,2019-11-01T02:58:00+00:00,2019-11-01T05:58:00+00:00
9,FC,13.45,2019-11-01T05:39:00+00:00,2019-11-01T05:52:00+00:00
...,...,...,...,...
15178,FC,11.17,2019-12-16T00:00:00+00:00,2019-12-16T00:14:00+00:00
15185,FC,12.41,2019-12-22T08:02:00+00:00,2019-12-22T08:14:00+00:00
15187,FC,13.37,2020-01-04T22:38:00+00:00,2020-01-04T22:58:00+00:00
15188,FC,14.77,2019-12-22T13:10:00+00:00,2019-12-22T13:28:00+00:00


In [139]:
df_clean['출발예상시간']>df_clean['연결시작시간']

0         True
1         True
2         True
3         True
4         True
         ...  
15195     True
15196    False
15197     True
15198     True
15199     True
Length: 15200, dtype: bool

In [143]:
mask = (df_clean['출발예상시간']>df_clean['연결시작시간'])==False
df_clean[mask]

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
7,st-00,st-00_evse-00,FC,Available,100,n,n,25.05,24.95,0,...,2019-11-19T08:41:00+00:00,2019-11-20T16:05:00+00:00,2019-11-20T16:07:00+00:00,2019-11-20T16:32:00+00:00,2019-11-20T16:41:00+00:00,None,90,20.3,25.4,5.10
8,st-00,st-00_evse-00,FC,Available,100,n,n,27.52,28.47,0,...,2019-11-20T16:41:00+00:00,2019-11-24T00:31:00+00:00,2019-11-24T00:37:00+00:00,2019-11-24T01:05:00+00:00,2019-11-24T01:14:00+00:00,None,90,36.5,29.6,6.52
14,st-00,st-00_evse-00,FC,Available,100,n,n,8.91,8.65,0,...,2019-12-06T20:12:00+00:00,2019-12-09T03:35:00+00:00,2019-12-09T03:37:00+00:00,2019-12-09T03:49:00+00:00,2019-12-09T04:02:00+00:00,None,50,46.6,21.4,5.92
16,st-00,st-00_evse-00,FC,Available,100,n,n,11.27,11.23,0,...,2019-12-12T13:16:00+00:00,2019-12-14T20:42:00+00:00,2019-12-14T21:11:00+00:00,2019-12-14T21:18:00+00:00,2019-12-14T21:32:00+00:00,None,90,54.0,15.0,7.02
23,st-00,st-00_evse-00,FC,Available,100,n,n,12.39,12.67,0,...,2019-12-26T22:56:00+00:00,2019-12-29T06:18:00+00:00,2019-12-29T06:38:00+00:00,2019-12-29T06:48:00+00:00,2019-12-29T07:02:00+00:00,None,90,38.1,16.9,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15150,st-19,st-19_evse-02,FC,Available,200,n,y,0.00,0.00,0,...,2020-09-17T21:24:00+00:00,2020-09-20T03:20:00+00:00,None,None,2020-09-20T03:41:00+00:00,None,75,12.7,5.0,4.01
15151,st-19,st-19_evse-02,FC,Available,200,n,y,4.88,5.05,0,...,2020-09-20T03:41:00+00:00,2020-09-20T09:59:00+00:00,2020-09-20T10:02:00+00:00,2020-09-20T10:08:00+00:00,2020-09-20T10:13:00+00:00,None,60,28.9,9.7,6.42
15158,st-19,st-19_evse-02,FC,Available,200,n,y,7.98,8.15,0,...,2020-10-06T00:54:00+00:00,2020-10-09T06:34:00+00:00,2020-10-09T07:04:00+00:00,2020-10-09T07:09:00+00:00,2020-10-09T07:21:00+00:00,None,75,57.5,8.6,5.35
15178,st-19,st-19_evse-02,FC,Available,200,n,n,10.84,11.03,0,...,2020-11-20T11:44:00+00:00,2020-11-22T17:51:00+00:00,2020-11-22T18:11:00+00:00,2020-11-22T18:21:00+00:00,2020-11-22T18:28:00+00:00,None,60,19.5,20.8,7.50


In [141]:
df_clean['마지막충전종료시간']<df_clean['연결시작시간']

0        True
1        True
2        True
3        True
4        True
         ... 
15195    True
15196    True
15197    True
15198    True
15199    True
Length: 15200, dtype: bool

In [142]:
df_clean['마지막충전종료시간']<df_clean['연결종료시간']

0        True
1        True
2        True
3        True
4        True
         ... 
15195    True
15196    True
15197    True
15198    True
15199    True
Length: 15200, dtype: bool

In [40]:
df_clean['연결종료시간']>df_clean['연결시작시간']

0        True
1        True
2        True
3        True
4        True
         ... 
15195    True
15196    True
15197    True
15198    True
15199    True
Length: 15200, dtype: bool

In [41]:
mask = (df_clean['충전종료시간']>df_clean['충전시작시간']) == False
df_clean[mask]

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
169,st-15,st-15_evse-05,SC,Available,10,y,y,0.00,0.0,6,...,2019-11-01T18:05:00+00:00,2019-11-01T19:48:00+00:00,None,None,2019-11-01T21:31:00+00:00,None,75,50.9,6.6,6.73
204,st-12,st-12_evse-04,FC,Available,200,n,n,8.58,0.0,0,...,2019-11-01T05:41:00+00:00,2019-11-01T08:26:00+00:00,None,None,2019-11-01T09:16:00+00:00,None,50,28.7,11.5,5.59
422,st-12,st-12_evse-05,SC,Available,30,y,n,0.00,0.0,9,...,2019-11-01T20:43:00+00:00,2019-11-01T21:33:00+00:00,None,None,2019-11-01T23:26:00+00:00,None,90,26.6,5.0,5.27
426,st-10,st-10_evse-02,SC,Available,10,n,y,0.00,0.0,0,...,2019-11-02T13:41:00+00:00,2019-11-02T15:30:00+00:00,None,None,2019-11-02T17:21:00+00:00,None,90,42.9,5.0,4.89
819,st-12,st-12_evse-01,FC,Available,100,n,y,0.00,0.0,0,...,2019-11-02T09:08:00+00:00,2019-11-02T10:26:00+00:00,None,None,2019-11-02T11:28:00+00:00,None,100,15.3,5.0,6.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14785,st-18,st-18_evse-01,SC,Available,50,n,n,3.25,0.0,0,...,2019-12-04T04:36:00+00:00,2019-12-04T06:57:00+00:00,None,None,2019-12-04T10:12:00+00:00,None,50,11.8,5.0,5.57
14841,st-17,st-17_evse-00,SC,Available,10,y,n,0.00,0.0,13,...,2019-12-06T11:58:00+00:00,2019-12-06T14:24:00+00:00,None,None,2019-12-06T17:08:00+00:00,None,60,60.0,5.0,4.74
15006,st-16,st-16_evse-05,SC,Available,50,n,n,9.60,0.0,0,...,2019-12-05T00:42:00+00:00,2019-12-05T01:17:00+00:00,None,None,2019-12-05T04:10:00+00:00,None,75,34.2,18.3,3.75
15114,st-18,st-18_evse-01,SC,Available,50,n,n,0.00,0.0,0,...,2019-12-05T05:59:00+00:00,2019-12-05T08:31:00+00:00,None,None,2019-12-05T11:37:00+00:00,None,50,8.5,5.0,4.89


In [42]:
ts = df_clean[mask]
ts['연결시작시간']<ts['연결종료시간']

169      True
204      True
422      True
426      True
819      True
         ... 
14785    True
14841    True
15006    True
15114    True
15127    True
Length: 104, dtype: bool

In [177]:
df_clean[df_clean['충전소위치'] == 'st-12']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
7600,st-12,st-12_evse-00,SC,Available,30,n,n,9.35,9.62,0,...,2019-11-01T00:00:00+00:00,2019-11-02T14:29:00+00:00,2019-11-02T14:45:00+00:00,2019-11-02T16:20:00+00:00,2019-11-02T16:28:00+00:00,None,50,12.5,18.0,5.16
7601,st-12,st-12_evse-00,SC,Available,30,y,n,12.42,12.08,1,...,2019-11-02T16:28:00+00:00,2019-11-03T17:13:00+00:00,2019-11-03T17:29:00+00:00,2019-11-03T19:34:00+00:00,2019-11-03T19:46:00+00:00,2019-11-03T19:28:00+00:00,100,31.7,10.0,4.72
7602,st-12,st-12_evse-00,SC,Available,30,n,y,16.87,17.03,0,...,2019-11-03T19:46:00+00:00,2019-11-05T17:23:00+00:00,2019-11-05T17:27:00+00:00,2019-11-05T20:06:00+00:00,2019-11-05T20:15:00+00:00,2019-11-05T20:49:00+00:00,50,47.2,31.4,6.64
7603,st-12,st-12_evse-00,SC,Available,30,n,y,12.74,13.08,0,...,2019-11-05T20:15:00+00:00,2019-11-07T00:03:00+00:00,2019-11-07T00:08:00+00:00,2019-11-07T02:04:00+00:00,2019-11-07T02:15:00+00:00,2019-11-07T02:32:00+00:00,90,8.8,15.4,5.75
7604,st-12,st-12_evse-00,SC,Available,30,y,n,13.12,13.02,2,...,2019-11-07T02:15:00+00:00,2019-11-09T03:44:00+00:00,2019-11-09T04:05:00+00:00,2019-11-09T06:28:00+00:00,2019-11-09T06:36:00+00:00,2019-11-09T06:56:00+00:00,100,33.6,9.6,4.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,st-12,st-12_evse-05,SC,Available,30,n,n,21.69,22.14,0,...,2020-11-07T12:20:00+00:00,2020-11-09T04:42:00+00:00,2020-11-09T04:53:00+00:00,2020-11-09T07:53:00+00:00,2020-11-09T07:59:00+00:00,2020-11-09T08:15:00+00:00,75,23.8,25.6,5.48
8796,st-12,st-12_evse-05,SC,Available,30,y,n,19.26,19.71,15,...,2020-11-09T07:59:00+00:00,2020-11-10T19:25:00+00:00,2020-11-10T19:31:00+00:00,2020-11-10T22:24:00+00:00,2020-11-10T22:37:00+00:00,2020-11-10T22:30:00+00:00,100,6.4,17.0,5.53
8797,st-12,st-12_evse-05,SC,Available,30,y,y,0.00,0.00,12,...,2020-11-10T22:37:00+00:00,2020-11-14T05:16:00+00:00,None,None,2020-11-14T05:55:00+00:00,None,90,30.4,5.0,6.01
8798,st-12,st-12_evse-05,SC,Available,30,y,y,28.17,28.57,11,...,2020-11-14T05:55:00+00:00,2020-11-16T22:57:00+00:00,2020-11-16T23:26:00+00:00,2020-11-17T02:26:00+00:00,2020-11-17T02:30:00+00:00,2020-11-17T02:43:00+00:00,50,60.0,58.9,5.39


In [74]:
df_clean[df_clean['충전기이름']=='st-00_evse-01']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
62,st-00,st-00_evse-01,FC,Available,50,n,n,13.48,13.90,0,...,2019-11-01T00:00:00+00:00,2019-11-01T04:34:00+00:00,2019-11-01T04:38:00+00:00,2019-11-01T04:53:00+00:00,2019-11-01T04:56:00+00:00,2019-11-01T05:11:00+00:00,50,32.1,23.8,5.47
168,st-00,st-00_evse-01,FC,Available,50,n,n,24.72,24.20,0,...,2019-11-01T04:56:00+00:00,2019-11-01T09:53:00+00:00,2019-11-01T09:59:00+00:00,2019-11-01T10:23:00+00:00,2019-11-01T10:34:00+00:00,2019-11-01T10:15:00+00:00,100,58.3,27.9,5.99
257,st-00,st-00_evse-01,FC,Available,50,n,n,7.75,8.05,0,...,2019-11-01T10:34:00+00:00,2019-11-01T14:58:00+00:00,2019-11-01T15:02:00+00:00,2019-11-01T15:12:00+00:00,2019-11-01T15:16:00+00:00,2019-11-01T15:08:00+00:00,90,30.4,5.0,5.27
319,st-00,st-00_evse-01,FC,Available,50,n,n,14.83,14.65,0,...,2019-11-01T15:16:00+00:00,2019-11-01T19:38:00+00:00,2019-11-01T19:58:00+00:00,2019-11-01T20:12:00+00:00,2019-11-01T20:15:00+00:00,2019-11-01T20:04:00+00:00,100,13.8,19.1,4.32
391,st-00,st-00_evse-01,FC,Available,50,n,n,19.45,20.09,0,...,2019-11-01T20:15:00+00:00,2019-11-02T00:55:00+00:00,2019-11-02T01:05:00+00:00,2019-11-02T01:21:00+00:00,2019-11-02T01:30:00+00:00,2019-11-02T02:17:00+00:00,50,22.3,41.3,5.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14738,st-00,st-00_evse-01,FC,Available,50,n,n,17.08,16.93,0,...,2019-12-12T01:54:00+00:00,2019-12-12T06:47:00+00:00,2019-12-12T07:07:00+00:00,2019-12-12T07:19:00+00:00,2019-12-12T07:34:00+00:00,2019-12-12T08:07:00+00:00,60,30.5,29.0,5.83
14798,st-00,st-00_evse-01,FC,Available,50,n,y,9.96,9.86,0,...,2019-12-12T07:34:00+00:00,2019-12-12T11:42:00+00:00,2019-12-12T11:53:00+00:00,2019-12-12T12:04:00+00:00,2019-12-12T12:13:00+00:00,2019-12-12T12:23:00+00:00,100,60.0,10.0,5.96
14974,st-00,st-00_evse-01,FC,Available,50,n,y,31.50,30.76,0,...,2019-12-12T12:13:00+00:00,2019-12-12T16:40:00+00:00,2019-12-12T16:48:00+00:00,2019-12-12T17:18:00+00:00,2019-12-12T17:33:00+00:00,None,90,39.6,36.6,6.29
15028,st-00,st-00_evse-01,FC,Available,50,n,y,20.06,20.06,0,...,2019-12-12T17:33:00+00:00,2019-12-12T21:38:00+00:00,2019-12-12T21:46:00+00:00,2019-12-12T22:09:00+00:00,2019-12-12T22:21:00+00:00,2019-12-12T22:24:00+00:00,90,37.6,26.9,5.05


In [ ]:
df_clean[df_clean['충전소위치']=='st-14']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
10000,st-14,st-14_evse-00,SC,Available,30,y,n,10.04,10.28,8,...,2019-11-01T00:00:00+00:00,2019-11-01T05:47:00+00:00,2019-11-01T06:15:00+00:00,2019-11-01T09:15:00+00:00,2019-11-01T09:28:00+00:00,None,50,40.3,20.9,4.21
10001,st-14,st-14_evse-00,SC,Available,30,y,n,23.47,23.01,29,...,2019-11-01T09:28:00+00:00,2019-11-01T15:00:00+00:00,2019-11-01T15:21:00+00:00,2019-11-01T18:21:00+00:00,2019-11-01T18:33:00+00:00,2019-11-01T18:45:00+00:00,100,39.6,22.5,5.87
10002,st-14,st-14_evse-00,SC,Available,30,y,y,19.05,18.97,8,...,2019-11-01T18:33:00+00:00,2019-11-02T00:08:00+00:00,2019-11-02T00:35:00+00:00,2019-11-02T03:35:00+00:00,2019-11-02T03:42:00+00:00,2019-11-02T03:42:00+00:00,60,23.7,31.0,6.00
10003,st-14,st-14_evse-00,SC,Available,30,n,y,12.64,13.00,0,...,2019-11-02T03:42:00+00:00,2019-11-02T10:30:00+00:00,2019-11-02T10:31:00+00:00,2019-11-02T12:22:00+00:00,2019-11-02T12:26:00+00:00,2019-11-02T12:43:00+00:00,100,8.4,11.5,4.77
10004,st-14,st-14_evse-00,SC,Available,30,n,y,19.50,19.63,0,...,2019-11-02T12:26:00+00:00,2019-11-02T18:23:00+00:00,2019-11-02T18:38:00+00:00,2019-11-02T21:38:00+00:00,2019-11-02T21:46:00+00:00,2019-11-02T21:52:00+00:00,50,55.5,34.9,6.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,st-14,st-14_evse-04,SC,Available,10,n,n,25.88,25.90,0,...,2020-04-20T23:12:00+00:00,2020-04-21T05:57:00+00:00,2020-04-21T06:09:00+00:00,2020-04-21T09:09:00+00:00,2020-04-21T09:20:00+00:00,None,50,9.8,47.4,5.69
10996,st-14,st-14_evse-04,SC,Available,10,n,n,7.73,7.85,0,...,2020-04-21T09:20:00+00:00,2020-04-21T15:48:00+00:00,2020-04-21T15:51:00+00:00,2020-04-21T17:37:00+00:00,2020-04-21T17:40:00+00:00,2020-04-21T19:03:00+00:00,100,7.9,7.2,5.40
10997,st-14,st-14_evse-04,SC,Available,10,y,n,17.44,17.92,10,...,2020-04-24T17:40:00+00:00,2020-04-25T00:17:00+00:00,2020-04-25T00:45:00+00:00,2020-04-25T03:45:00+00:00,2020-04-25T03:51:00+00:00,2020-04-25T03:46:00+00:00,75,25.9,20.5,4.15
10998,st-14,st-14_evse-04,SC,Available,10,n,n,17.78,17.70,0,...,2020-04-25T03:51:00+00:00,2020-04-25T11:18:00+00:00,2020-04-25T11:35:00+00:00,2020-04-25T14:35:00+00:00,2020-04-25T14:47:00+00:00,2020-04-25T15:29:00+00:00,90,33.4,20.8,7.50


In [ ]:
df_clean[df_clean['충전기이름']=='st-14_evse-04']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
10800,st-14,st-14_evse-04,SC,Available,10,n,y,25.21,25.05,0,...,2019-11-01T00:00:00+00:00,2019-11-03T03:12:00+00:00,2019-11-03T03:14:00+00:00,2019-11-03T06:14:00+00:00,2019-11-03T06:29:00+00:00,2019-11-03T08:02:00+00:00,60,14.3,41.9,5.49
10801,st-14,st-14_evse-04,SC,Available,10,n,n,23.99,24.53,0,...,2019-11-03T06:29:00+00:00,2019-11-05T09:20:00+00:00,2019-11-05T09:48:00+00:00,2019-11-05T12:48:00+00:00,2019-11-05T12:56:00+00:00,2019-11-05T13:16:00+00:00,100,18.4,21.9,4.30
10802,st-14,st-14_evse-04,SC,Available,10,y,n,12.88,13.29,14,...,2019-11-05T12:56:00+00:00,2019-11-06T15:40:00+00:00,2019-11-06T15:59:00+00:00,2019-11-06T17:51:00+00:00,2019-11-06T18:00:00+00:00,None,100,31.1,14.0,5.76
10803,st-14,st-14_evse-04,SC,Available,10,n,n,16.63,17.04,0,...,2019-11-06T18:00:00+00:00,2019-11-08T20:38:00+00:00,2019-11-08T20:43:00+00:00,2019-11-08T23:43:00+00:00,2019-11-08T23:48:00+00:00,2019-11-09T00:02:00+00:00,100,20.8,12.8,6.45
10804,st-14,st-14_evse-04,SC,Available,10,n,n,24.88,24.83,0,...,2019-11-08T23:48:00+00:00,2019-11-11T03:07:00+00:00,2019-11-11T03:20:00+00:00,2019-11-11T06:20:00+00:00,2019-11-11T06:25:00+00:00,2019-11-11T06:21:00+00:00,50,8.1,48.1,4.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,st-14,st-14_evse-04,SC,Available,10,y,y,11.84,11.43,12,...,2020-12-21T13:20:00+00:00,2020-12-23T16:14:00+00:00,2020-12-23T16:24:00+00:00,2020-12-23T18:24:00+00:00,2020-12-23T18:38:00+00:00,2020-12-23T18:49:00+00:00,75,28.8,14.0,4.88
10996,st-14,st-14_evse-04,SC,Available,10,y,y,9.51,9.69,27,...,2020-12-23T18:38:00+00:00,2020-12-26T21:22:00+00:00,2020-12-26T21:24:00+00:00,2020-12-26T22:48:00+00:00,2020-12-26T22:50:00+00:00,2020-12-26T22:54:00+00:00,50,8.8,20.6,6.99
10997,st-14,st-14_evse-04,SC,Available,10,n,n,24.77,24.11,0,...,2020-12-26T22:50:00+00:00,2020-12-28T02:07:00+00:00,2020-12-28T02:25:00+00:00,2020-12-28T05:25:00+00:00,2020-12-28T05:27:00+00:00,2020-12-28T06:08:00+00:00,90,25.5,29.1,5.14
10998,st-14,st-14_evse-04,SC,Available,10,y,n,13.77,13.51,18,...,2020-12-28T05:27:00+00:00,2020-12-30T09:15:00+00:00,2020-12-30T09:38:00+00:00,2020-12-30T12:07:00+00:00,2020-12-30T12:15:00+00:00,2020-12-30T12:16:00+00:00,50,29.2,30.6,5.57


① 방전 지원 여부 → 방전량/요청방전량 가능 여부
     └─ 'n'이면 둘 다 반드시 0
     
② 요청충전량 == 0 → 충전량은 반드시 0 (예외 시 '충전 불필요' 시나리오로 분리)

③ 충전기 상태 → 충전량에 영향
     └─ 'Faulted'이면 충전량=0
     └─ 'Available'인데 충전량 > 0 이면 충돌

④ 충전 시작/종료 시간 → 시간 역전 불가
     └─ 종료 > 시작 > 연결 > 이전 종료

⑤ 시작/종료 배터리 퍼센트 + 충전량 → 일관성 필요
     └─ 종료퍼센트 < 시작퍼센트면 충전량 = 0 or 음수

⑥ 출발예상시간 → 필수 입력 아님
     └─ 사용자 입력 없는 경우 `NaT` 또는 `None`

⑦ 충전기 타입 → 방전지원여부 제약
     └─ 대부분 FC는 방전 미지원 → y 불가



⚡ 충전 상태 기반 패턴 (Status-driven)
- 정상 충전 완료
- 연결 후 충전 실패
- 충전 중 연결 종료
- 중간에 사용자에 의해 중단
- 충전 시작 지연
- 급속 충전 중단
- 완속 충전 연장
- 충전 불필요 (베터리 충분)

🕒 시간 흐름 패턴 (Time-driven) - 일부 반영
- 야간 충전 (22시 ~ 06시)
- 오전 집중 충전 (출근 직전)
- 낮 시간 대기 후 충전
- 급속 충전으로 출발시간 전 도달
- 출발예상시간보다 늦게 충전 종료
- 연결시간 긴데 충전시간 짧음
- 충전기 대기 시간 포함된 시나리오
- 출발예상시간이 예측 충전과 연동
- 베터리 용량 적을 때 반복 충전
- 마지막충전종료시간과 출발시간 간격 비교

🔋 배터리/충전량 기반 패턴 (Energy-driven) -- 아직 반영하지 않음
- 시작 베터리 10%, 종료 90% (충전량 충분)
- 시작 30%, 종료 60% (부분충전)
- 시작 80%, 종료 100% (최적화)
- 종료 후 방전량 존재
- 요청방전량 > 실제방전량
- 방전 후 재충전
- 요청충전량 ≠ 실제충전량 (시간 부족 또는 제약)
- 고용량 충전기에서 30분 만에 완충
- 낮은 연비일 때 충전량 많음
- 시작베터리 높지만 방전 요구 있음

📍 위치/장비 타입 기반 패턴 (Location/type-driven)  -일부 반영
- FC 타입에서 빠른 완충
- SC 타입에서 오래 연결
- 용량이 작지만 빈도 높은 충전기
- FC 충전기에서 방전은 미지원
- SC 충전기에서 연비 반영 충전
- 동일 위치에서 반복 사용되는 사용자
- 충전기이름별 성능 차이 시뮬레이션

🔁 특수 상황 패턴 (Exception-driven) 
- 방전지원 차량만 가능한 시나리오
- 마지막충전종료시간이 기록 누락됨
- 연결시간은 있으나 실제 충전 없음
- 요청 충전량이 0인데 충전됨
- 예측 출발 시간보다 빠르게 종료
